### Assignment 9: AI Planner Using A* for Task Scheduling
Objective: Use A* Search to optimize task scheduling.

Problem Statement: A set of tasks with dependencies and durations needs to be scheduled to minimize total time.

#### Tasks:
* Represent tasks and dependencies as a directed graph.
* Use A* Search when the heuristic estimates the remaining task's duration.
* Compare results with a greedy algorithm.

In [1]:
import heapq
from collections import defaultdict

def compute_indegree(tasks, deps):
    indegree = {t: 0 for t in tasks}
    for t, prereqs in deps.items():
        for p in prereqs:
            indegree[t] += 1
    return indegree

class State:
    __slots__ = ('time', 'done', 'procs')
    def __init__(self, time, done, procs):
        self.time = time
        self.done = frozenset(done)
        self.procs = tuple(procs)

    def __lt__(self, other):
        return self.time < other.time

    def key(self):
        return (self.done, self.procs)


def heuristic(remaining_dur, m):
    # admissible: total remaining work divided by processors
    return sum(remaining_dur) / m if m > 0 else float('inf')


def a_star_schedule(tasks, durations, deps, m):
    # tasks: list of task IDs
    # durations: dict task->duration
    # deps: dict task->list of prerequisites
    # m: number of processors
    indegree_init = compute_indegree(tasks, deps)
    dependents = defaultdict(list)
    for t, prereqs in deps.items():
        for p in prereqs:
            dependents[p].append(t)
    all_durs = durations.copy()
    open_heap = []
    visited = {}
    start = State(0, [], [0]*m)
    start_h = heuristic(all_durs.values(), m)
    heapq.heappush(open_heap, (start_h, start))
    visited[start.key()] = 0
    while open_heap:
        f, s = heapq.heappop(open_heap)
        g = s.time
        if len(s.done) == len(tasks):
            return g
        indegree = indegree_init.copy()
        for d in s.done:
            for nb in dependents[d]:
                indegree[nb] -= 1
        ready = [t for t in tasks if t not in s.done and indegree[t] == 0]
        for t in ready:
            dur = durations[t]
            for i in range(m):
                procs = list(s.procs)
                start_time = max(s.time, procs[i])
                finish = start_time + dur
                procs[i] = finish
                new_done = set(s.done)
                new_done.add(t)
                new_time = min(procs)
                new_state = State(new_time, new_done, procs)
                key = new_state.key()
                cost = max(new_state.procs)
                if key not in visited or cost < visited[key]:
                    visited[key] = cost
                    rem = [all_durs[x] for x in tasks if x not in new_done]
                    h = heuristic(rem, m)
                    heapq.heappush(open_heap, (cost + h, new_state))
    return None

def greedy_schedule(tasks, durations, deps, m):
    indegree = compute_indegree(tasks, deps)
    dependents = defaultdict(list)
    for t, prereqs in deps.items():
        for p in prereqs:
            dependents[p].append(t)
    time = 0
    procs = [0]*m
    done = set()
    ready = [t for t in tasks if indegree[t] == 0]
    while ready:
        for i in sorted(range(m), key=lambda x: procs[x]):
            if not ready:
                break
            t = ready.pop(0)
            start_time = max(time, procs[i])
            procs[i] = start_time + durations[t]
            done.add(t)
            for nb in dependents[t]:
                indegree[nb] -= 1
                if indegree[nb] == 0:
                    ready.append(nb)
        time = min(procs)
    return max(procs)

tasks = ['A', 'B', 'C', 'D', 'E', 'F']
durations = {'A': 3, 'B': 2, 'C': 4, 'D': 2, 'E': 1, 'F': 3}
deps = {
    'B': ['A'],
    'C': ['A'],
    'D': ['B', 'C'],
    'E': ['C'],
    'F': ['D', 'E']
}
m = 2
print("A* makespan:", a_star_schedule(tasks, durations, deps, m))
print("Greedy makespan:", greedy_schedule(tasks, durations, deps, m))

A* makespan: 7
Greedy makespan: 9
